<a id="toc"></a>
# Ray tracing plots with barc4plots
***

Contents :
1. [Energy resolution](#Eres)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '22/OCT/2024'
__changed__ = '06/NOV/2024'

import re
import sys
from array import array
from glob import glob
from copy import deepcopy
import barc4xoc.beam as xbm
from barc4xoc.misc import moving_average
import numpy as np
from barc4plots.barc4plots import PlotManager
from barc4xoc.aux_pyoptix import read_beam_from_csv, read_resolution_curve
from barc4xoc.grating import treat_energy_resolution
from barc4xoc.plots import plot_beam
# checking we are using the correct python:
print(sys.executable)
print(sys.version)
# %matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
energy_resolution = False

mono_exit_slit = False
mono_exit_slit_II = False

wolter_A_cromag = False
wolter_A_mk2T = False

KB_A_cromag = False
KB_A_mk2T = False

wolter_B_cromag = False
wolter_B_mk2T = False

KB_B_cromag = False
KB_B_mk2T = False

dispersion_2d = True

pattern = r"[^\\]+(?=\.[^.]+$)"

<a id="Eres"></a>
## Energy resolution
[Back to the top](#toc)

In [ ]:
e0, ei, ef, dE = 0, 250, 5000, 5

In [ ]:
if energy_resolution:
    DRC_1600 = treat_energy_resolution(read_resolution_curve(r".\results\pyoptix\mono\deimos_resolution_curves_G1600_5000_rays.csv"), deltaE=500, fit=False)

    scan = [10, 20, 30, 50, 80]
    img = PlotManager()
    img.additional_info('resolving power', "energy [eV]",  "E/$\Delta$E ($\\times$1E3)", xmin=e0, xmax=ef+(ei-e0), ymin=5, ymax=30, 
                        sort_ax=False, sort_ax_lim=False).aesthetics(400, True, True, 0, 1, True, 4)

    for i, cff in enumerate(scan):
        img.image, img.x = DRC_1600[f"U52_M1B_G1600_mono_cff_0.{cff}"]/1E3, DRC_1600["energy"]
        img.info_1d_plot(i, '', 1, ".", alpha=0.1).plot_1d(enable=False, hold=not(i==0))

    for i, cff in enumerate(scan):
        img.image, img.x = DRC_1600[f"U52_M1B_G1600_mono_cff_0.{cff}_fit_moving_avg"]/1E3, DRC_1600["energy"]
        img.info_1d_plot(i, f'cff = 0.{cff}', 1, "-", alpha=1).plot_1d(enable=(i==len(scan)-1), hold=True)

In [ ]:
if energy_resolution:
    DRC_1600 = treat_energy_resolution(read_resolution_curve(r".\results\pyoptix\deimos_II_resolution_curves_G1600_5000_rays.csv"), deltaE=500, fit=False)

    scan = [10, 20, 30, 50, 80]
    img = PlotManager()
    img.additional_info('resolving power', "energy [eV]",  "E/$\Delta$E ($\\times$1E3)", xmin=e0, xmax=ef+(ei-e0), ymin=5, ymax=35, 
                        sort_ax=False, sort_ax_lim=False).aesthetics(400, True, True, 0, 1, True, 4)

    for i, cff in enumerate(scan):
        img.image, img.x = DRC_1600[f"SII_M1B_G1600_mono_cff_0.{cff}"]/1E3, DRC_1600["energy"]
        img.info_1d_plot(i, '', 1, ".", alpha=0.1).plot_1d(enable=False, hold=not(i==0))

    for i, cff in enumerate(scan):
        img.image, img.x = DRC_1600[f"SII_M1B_G1600_mono_cff_0.{cff}_fit_moving_avg"]/1E3, DRC_1600["energy"]
        img.info_1d_plot(i, f'cff = 0.{cff}', 1, "-", alpha=1).plot_1d(enable=(i==len(scan)-1), hold=True)

In [ ]:
if energy_resolution:
    DRC_2400 = treat_energy_resolution(read_resolution_curve(r".\results\pyoptix\mono\deimos_resolution_curves_G2400_5000_rays.csv"), deltaE=500, fit=False)

    img = PlotManager()
    img.additional_info('resolving power', "energy [eV]",  "E/$\Delta$E ($\\times$1E3)", xmin=e0, xmax=ef+(ei-e0), ymin=5, ymax=30, 
                        sort_ax=False, sort_ax_lim=False).aesthetics(400, True, True, 0, 1, True, 4)
    img.image, img.x = DRC_2400["U52_M1C_G2400_mono_omega_0.01"]/1E3, DRC_2400["energy"]
    img.info_1d_plot(3, '', 1, ".", alpha=0.1).plot_1d(enable=False)

    img.image, img.x = DRC_2400["U52_M1C_G2400_mono_omega_0.01_fit_moving_avg"]/1E3, DRC_2400["energy"]
    img.info_1d_plot(3, 'HE/G2400/$\Omega$=0.692', 1, "-", alpha=1).plot_1d(enable=True, hold=True)

In [ ]:
if energy_resolution:
    DRC_2400 = treat_energy_resolution(read_resolution_curve(r".\results\pyoptix\deimos_II_resolution_curves_G2400_5000_rays.csv"), deltaE=500, fit=False)

    img = PlotManager()
    img.additional_info('resolving power', "energy [eV]",  "E/$\Delta$E ($\\times$1E3)", xmin=e0, xmax=ef+(ei-e0), ymin=5, ymax=30, 
                        sort_ax=False, sort_ax_lim=False).aesthetics(400, True, True, 0, 1, True, 4)
    img.image, img.x = DRC_2400["SII_M1C_G2400_mono_omega_0.01"]/1E3, DRC_2400["energy"]
    img.info_1d_plot(3, '', 1, ".", alpha=0.1).plot_1d(enable=False)

    img.image, img.x = DRC_2400["SII_M1C_G2400_mono_omega_0.01_fit_moving_avg"]/1E3, DRC_2400["energy"]
    img.info_1d_plot(3, 'HE/G2400/$\Omega$=0.692', 1, "-", alpha=1).plot_1d(enable=True, hold=True)

<a id="mono_exit_slit"></a>
## Mono exit slit
[Back to the top](#toc)

### Deimos

In [ ]:
def plot_scan(configs, labels, type_of_plot, prange, **kwargs):

    colours = kwargs.get('colours', range(len(configs)))

    img = PlotManager()
    img.additional_info(type_of_plot[0], "energy [eV]",  type_of_plot[1], 
                        xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3], 
                        sort_ax=False, sort_ax_lim=False).aesthetics(400, True, True, 0, 1, True, 4)
    for i, (config, label) in enumerate(zip(configs, labels)):

        group_name = list(config.keys())[0]

        if len(type_of_plot)>4:
            stat = deepcopy(config[group_name][type_of_plot[2]][type_of_plot[3]][type_of_plot[4]])
            if type_of_plot[4] == 'std' or type_of_plot[4] == 'fwhm':
                stat *= 1E6
        else:
            stat = deepcopy(config[group_name][type_of_plot[2]][type_of_plot[3]])

        stat[stat==0] = np.nan
        img.image, img.x = stat, config[group_name]["scan"]
        img.info_1d_plot(colours[i], label, 1, "-").plot_1d(enable=(i==len(configs)-1), hold=(i!=0))

In [ ]:
if mono_exit_slit:
    
    stats_LE_B = xbm.load_stats_from_csv(r".\results\pyoptix\mono\U52_M1B_G1600_mono_@_mono_foc_ver_energy_scan_20000_rays_stats.csv")
    stats_LE_C = xbm.load_stats_from_csv(r".\results\pyoptix\mono\U52_M1C_G1600_mono_@_mono_foc_ver_energy_scan_20000_rays_stats.csv")
    stats_HE_C = xbm.load_stats_from_csv(r".\results\pyoptix\mono\U52_M1C_G2400_mono_@_mono_foc_ver_energy_scan_20000_rays_stats.csv")

    stats_LE_B_SII = xbm.load_stats_from_csv(r".\results\pyoptix\mono\SII_M1B_G1600_mono_@_mono_foc_ver_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(r".\results\pyoptix\mono\SII_M1C_G1600_mono_@_mono_foc_ver_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(r".\results\pyoptix\mono\SII_M1C_G2400_mono_@_mono_foc_ver_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B, stats_LE_B_SII, stats_LE_C, stats_LE_C_SII, stats_HE_C, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/G1600/cff=0.2', '(small $\epsilon$)',
            'LE/2400/$\Omega$=0.692', '(small $\epsilon$)']

    colours = [0, 3, 1, 4, 2, 5]

In [ ]:
if mono_exit_slit:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 20]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.12, 0.12]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 100]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    
    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 175, 625]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.12, 0.12]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 45, 165]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

In [ ]:
if mono_exit_slit and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\mono\*_M1B_G1600*_foc_ver_E350eV_20000*.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-2500, 2500, -800, 800])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-100, 100, -400, 400])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-2500, 2500, -100, 100])

In [ ]:
if mono_exit_slit and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\mono\*_M1C_G1600*_foc_ver_E1000eV_20000*.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-2500, 2500, -800, 800])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-100, 100, -400, 400])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-2500, 2500, -100, 100])

In [ ]:
if mono_exit_slit and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\mono\*_M1C_G2400*_foc_ver_E2000eV_20000*.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-2500, 2500, -800, 800])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-100, 100, -400, 400])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-2500, 2500, -100, 100])

### Deimos-II

In [ ]:
if mono_exit_slit_II and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\mono_II\SII_M1B_G1600_mono_@_mono_foc_ver_m3_*_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-55, 55, -275, 275])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-75, 75, -250, 250])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-75, 75, -55, 55])

In [ ]:
if mono_exit_slit_II and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\mono_II\SII_M1C_G1600_mono_@_mono_foc_ver_m3_*_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-55, 55, -275, 275])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-75, 75, -250, 250])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-75, 75, -55, 55])

In [ ]:
if mono_exit_slit_II and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\mono_II\SII_M1C_G2400_mono_@_mono_foc_ver_m3_*_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-55, 55, -275, 275])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-75, 75, -250, 250])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-75, 75, -55, 55])

<a id="exp-stat"></a>
## Experimental stations
[Back to the top](#toc)

### Wolter A - cromag

In [ ]:
focus = 'WLT_A'
exp = 'cromag'
directory = 'sample'

if wolter_A_cromag:
    source = 'U52'
    stats_LE_B = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B, stats_LE_B_SII, stats_LE_C, stats_LE_C_SII, stats_HE_C, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/2400/$\Omega$=0.692', '(small $\epsilon$)']

    colours = [0, 3, 1, 4, 2, 5]

In [ ]:
if wolter_A_cromag:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 12]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 7]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 200]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 25]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 3]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 100, 500]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


In [ ]:
if wolter_A_cromag and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1B_G1600_WLT_A_@_cromag_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-100, 100, -2000, 2000])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-40, 40, -800, 800])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-100, 100, -40, 40])

In [ ]:
if wolter_A_cromag and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G1600_WLT_A_@_cromag_E1000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-100, 100, -2000, 2000])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-40, 40, -800, 800])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-100, 100, -40, 40])

In [ ]:
if wolter_A_cromag and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G2400_WLT_A_@_cromag_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-100, 100, -2000, 2000])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-40, 40, -800, 800])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-100, 100, -40, 40])

### Wolter A - mk2T

In [ ]:
focus = 'WLT_A'
exp = 'mk2T'
directory = 'sample'

if wolter_A_mk2T:
    source = 'U52'
    stats_LE_B = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B, stats_LE_B_SII, stats_LE_C, stats_LE_C_SII, stats_HE_C, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/2400/$\Omega$=0.692', '(small $\epsilon$)']

    colours = [0, 3, 1, 4, 2, 5]

In [ ]:
if wolter_A_mk2T:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 350]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 200]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 200, 800]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.25, 0.25]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 100, 500]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

In [ ]:

if wolter_A_mk2T and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1B_G1600_WLT_A_@_mk2T_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-3500, 3500, -2000, 2000])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-1100, 1100, -600, 600])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-3200, 3200, -1100, 1100])
        

In [ ]:
if wolter_A_mk2T and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G1600_WLT_A_@_mk2T_E1000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-3500, 3500, -2000, 2000])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-1100, 1100, -600, 600])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-3200, 3200, -1100, 1100])

In [ ]:
if wolter_A_mk2T and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G2400_WLT_A_@_mk2T_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-3500, 3500, -2000, 2000])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-1100, 1100, -600, 600])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-3200, 3200, -1100, 1100])

### KB A - cromag

In [ ]:
focus = 'WLT_A'
exp = 'cromag'
directory = 'sample_II'

if KB_A_cromag:
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B_SII, stats_LE_C_SII, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2',
              'HE/G1600/cff=0.2',
              'HE/G2400/$\Omega$=0.692']
    colours = [0, 1, 2]

In [ ]:
if KB_A_cromag:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 22]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 4]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 22]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


In [ ]:
if KB_A_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1B_G1600_{focus}_@_{exp}_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_A_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G1600_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_A_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_A_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_A_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E3000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_A_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E4000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

### KB A - mk2T

In [ ]:
focus = 'WLT_A'
exp = 'mk2T'
directory = 'sample_II'

if KB_A_mk2T:
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B_SII, stats_LE_C_SII, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2',
              'HE/G1600/cff=0.2',
              'HE/G2400/$\Omega$=0.692']
    colours = [0, 1, 2]

In [ ]:
if KB_A_mk2T:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 210]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 210]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


In [ ]:
if KB_A_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1B_G1600_{focus}_@_{exp}_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_A_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G1600_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_A_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_A_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_A_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E3000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_A_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E4000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

### Wolter B - cromag

In [ ]:
focus = 'WLT_B'
exp = 'cromag'
directory = 'sample'

if wolter_B_cromag:
    source = 'U52'
    stats_LE_B = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B, stats_LE_B_SII, stats_LE_C, stats_LE_C_SII, stats_HE_C, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/2400/$\Omega$=0.692', '(small $\epsilon$)']

    colours = [0, 3, 1, 4, 2, 5]

In [ ]:
if wolter_B_cromag:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 25, 200]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.30]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.3]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 15, 120]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 150, 600]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.30]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.3]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 50, 320]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

In [ ]:
if wolter_B_cromag and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1B_G1600_WLT_B_@_cromag_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-2000, 2000, -1200, 1200])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-600, 600, -450, 450])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-2000, 2000, -600, 600])

In [ ]:
if wolter_B_cromag and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G1600_WLT_B_@_cromag_E1000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-2000, 2000, -1200, 1200])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-600, 600, -450, 450])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-2000, 2000, -600, 600])

In [ ]:
if wolter_B_cromag and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G2400_WLT_B_@_cromag_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-2000, 2000, -1200, 1200])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-600, 600, -450, 450])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-2000, 2000, -600, 600])

### Wolter B - mk2T

In [ ]:
focus = 'WLT_B'
exp = 'mk2T'
directory = 'sample'

if wolter_B_mk2T:
    source = 'U52'
    stats_LE_B = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B, stats_LE_B_SII, stats_LE_C, stats_LE_C_SII, stats_HE_C, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/G1600/cff=0.2', '(small $\epsilon$)',
            'HE/2400/$\Omega$=0.692', '(small $\epsilon$)']

    colours = [0, 3, 1, 4, 2, 5]

In [ ]:
if wolter_B_mk2T:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 17]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.30]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.3]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 15, 120]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"

    prange = [e0, ef+(ei-e0), None, None]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.30]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -0.3, 0.3]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 50, 320]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)

In [ ]:
if wolter_B_mk2T and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1B_G1600_WLT_B_@_mk2T_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-150, 150, -1200, 1200])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-60, 60, -450, 450])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-150, 150, -60, 60])

In [ ]:
if wolter_B_mk2T and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G1600_WLT_B_@_mk2T_E1000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-150, 150, -1200, 1200])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-60, 60, -450, 450])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-150, 150, -60, 60])

In [ ]:
if wolter_B_mk2T and dispersion_2d:
    simulations = sorted(glob(r".\results\pyoptix\sample\*_M1C_G2400_WLT_B_@_mk2T_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-150, 150, -1200, 1200])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-60, 60, -450, 450])
        plot_beam(beam, "size", direction='y', aspect_ratio=False, prange=[-150, 150, -60, 60])

### KB B - cromag

In [ ]:
focus = 'WLT_B'
exp = 'cromag'
directory = 'sample_II'

if KB_B_cromag:
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B_SII, stats_LE_C_SII, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2',
              'HE/G1600/cff=0.2',
              'HE/G2400/$\Omega$=0.692']
    colours = [0, 1, 2]

In [ ]:
if KB_B_cromag:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 210]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 210]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


In [ ]:
if KB_B_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1B_G1600_{focus}_@_{exp}_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_B_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G1600_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_B_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_B_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_B_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E3000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

In [ ]:
if KB_B_cromag and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E4000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-650, 650, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-650, 650, -650, 650])

### KB B - mk2T

In [ ]:
focus = 'WLT_B'
exp = 'mk2T'
directory = 'sample_II'

if KB_B_mk2T:
    source = 'SII'
    stats_LE_B_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1B_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_LE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G1600_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")
    stats_HE_C_SII = xbm.load_stats_from_csv(rf".\results\pyoptix\{directory}\{source}_M1C_G2400_{focus}_@_{exp}_energy_scan_20000_rays_stats.csv")

    configs = [stats_LE_B_SII, stats_LE_C_SII, stats_HE_C_SII]
    labels = ['LE/G1600/cff=0.2',
              'HE/G1600/cff=0.2',
              'HE/G2400/$\Omega$=0.692']
    colours = [0, 1, 2]

In [ ]:
if KB_B_mk2T:
    print(">>>>>>>>>>>>>>>>>>>>>> VERTICAL")
    direction = "Y"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 22]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 4]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "Y"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


    print(">>>>>>>>>>>>>>>>>>>>>> HORIZONTAL")
    direction = "X"
    caracteristic = "sigma"
    prange = [e0, ef+(ei-e0), 0, 22]
    type_of_plot = ["Beam size","rms (µm)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange,  colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Skewness", direction, caracteristic, "skew"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    prange = [e0, ef+(ei-e0), -1.5, 1.5]
    type_of_plot = ["Moments","Excess Kurtosis", direction, caracteristic, "kurtosis"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)
    direction = "X"
    caracteristic = "sigma_p"
    prange = [e0, ef+(ei-e0), 0, 110]
    type_of_plot = ["beam divergence","rms (µrad)", direction, caracteristic, "std"]
    plot_scan(configs, labels, type_of_plot, prange, colours=colours)


In [ ]:
if KB_B_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1B_G1600_{focus}_@_{exp}_E350eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_B_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G1600_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_B_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E1500eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_B_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E2000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_B_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E3000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])

In [ ]:
if KB_B_mk2T and dispersion_2d:
    simulations = sorted(glob(rf".\results\pyoptix\{directory}\*_M1C_G2400_{focus}_@_{exp}_E4000eV_20000_rays.csv"))
    for sim in simulations:
        print(re.search(pattern, sim).group(0))
        beam = xbm.read_beam_from_csv(sim)
        stats = xbm.get_beam_stats(beam, verbose=True)
        plot_beam(beam, "phase_space", direction='x', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "phase_space", direction='y', aspect_ratio=False, prange=[-80, 80, -320, 320])
        plot_beam(beam, "size", direction='y', aspect_ratio=True, prange=[-80, 80, -80, 80])